In [54]:
import numpy as np
import scipy
from sklearn.preprocessing import MinMaxScaler

dataset = np.loadtxt("/Users/Keshavamurthy/allfeatures1.txt", delimiter=" ");
print(dataset.shape)

np.random.shuffle(dataset)

X = dataset[:, :34]
Y = dataset[:, 34]
botnetDataset = dataset[dataset[:,34]==1, :]
normalDataset = dataset[dataset[:,34]==0, :]

botnetX = botnetDataset[:,:34]
botnetY = botnetDataset[:, 34]

normalX = normalDataset[:,:34]
normalY = normalDataset[:, 34]

scaler = MinMaxScaler(feature_range=(0, 1))
rescaledBotnetX = scaler.fit_transform(botnetX)
rescaledNormalX = scaler.fit_transform(normalX)
rescaledX = scaler.fit_transform(X)

valid_start_index = 0
valid_end_index = 500
test_start_index = 500
test_end_index = 1000
num_labels = 2

validation_dataset = np.concatenate((rescaledBotnetX[valid_start_index:valid_end_index,:], 
                                     rescaledNormalX[valid_start_index:valid_end_index,:]))
valid_labels = np.concatenate((botnetY[valid_start_index:valid_end_index], 
                                   normalY[valid_start_index:valid_end_index]))
valid_labels = (np.arange(num_labels) == valid_labels[:,None]).astype(np.float64)

test_dataset = np.concatenate((rescaledBotnetX[test_start_index:test_end_index,:], 
                               rescaledNormalX[test_start_index:test_end_index,:]))
test_labels = np.concatenate((botnetY[test_start_index:test_end_index], 
                             normalY[test_start_index:test_end_index]))
test_labels = (np.arange(num_labels) == test_labels[:,None]).astype(np.float64)

train_dataset = np.concatenate((rescaledBotnetX[test_end_index:,:], 
                               rescaledNormalX[test_end_index:,:]))
train_labels = np.concatenate((botnetY[test_end_index:], 
                             normalY[test_end_index:]))
train_labels = (np.arange(num_labels) == train_labels[:,None]).astype(np.float64)

print(validation_dataset.shape, valid_labels.shape)
print(test_dataset.shape, test_labels.shape)
print(train_dataset.shape, train_labels.shape)

(52580, 35)
(1000, 34) (1000, 2)
(1000, 34) (1000, 2)
(50580, 34) (50580, 2)


In [116]:
import tensorflow as tf

num_features = 34
valid_size = 1000
test_size = 1000
batch_size = 128
num_hidden_nodes = 8192

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float64, shape=(batch_size, num_features))
    tf_train_labels = tf.placeholder(tf.float64, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(validation_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_regul = tf.placeholder(tf.float64)
    
    # Variables.
    weights1 = tf.Variable(tf.truncated_normal([num_features, num_hidden_nodes], dtype=tf.float64), dtype=tf.float64)
    biases1 = tf.Variable(tf.zeros([num_hidden_nodes], dtype=tf.float64), dtype=tf.float64)
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels], dtype=tf.float64), dtype=tf.float64)
    biases2 = tf.Variable(tf.zeros([num_labels], dtype=tf.float64), dtype=tf.float64)
    
    # Training computation.
    lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    drop1 = tf.nn.dropout(lay1_train, 0.5)
    logits = tf.matmul(drop1, weights2) + biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
      beta_regul * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_prediction = tf.nn.softmax(tf.matmul(lay1_valid, weights2) + biases2)
    lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(lay1_test, weights2) + biases2)
  


In [117]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [118]:
def GetNextBotnetSet(rescaledX, Y):
    step = 4000
    prevStep = 0
    while step < rescaledX.shape[0]:
        yield rescaledX[prevStep:step, :], Y[prevStep:step]
        prevStep = step
        step = step + 4000
        
    return rescaledX[prevStep:, :], Y[prevStep:]

In [119]:
num_steps = 500
offset = int(test_end_index)
trainNormalX = rescaledNormalX[test_end_index:, :]
class_batch_size = int(batch_size)/2

with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    for step in range(num_steps):
        np.random.shuffle(trainNormalX)
        train_batch_data = np.concatenate((rescaledBotnetX[offset:(offset+class_batch_size),:], 
                               trainNormalX[:class_batch_size,:]))
        train_batch_labels = np.concatenate((botnetY[:class_batch_size], 
                             normalY[:class_batch_size]))
        train_batch_labels = (np.arange(num_labels) == train_batch_labels[:,None]).astype(np.float64)
        offset = offset + class_batch_size
        
        feed_dict = {tf_train_dataset : train_batch_data, 
                     tf_train_labels : train_batch_labels, 
                     beta_regul : 1e-3 
                    }
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, train_batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:11: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:12: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:13: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Minibatch loss at step 0: 118.170535
Minibatch accuracy: 45.3%
Validation accuracy: 50.0%
Minibatch loss at step 50: 108.578995
Minibatch accuracy: 90.6%
Validation accuracy: 87.0%
Minibatch loss at step 100: 103.404639
Minibatch accuracy: 89.1%
Validation accuracy: 87.6%
Minibatch loss at step 150: 98.108950
Minibatch accuracy: 92.2%
Validation accuracy: 90.0%
Minibatch loss at step 200: 94.973524
Minibatch accuracy: 85.9%
Validation accuracy: 90.1%
Minibatch loss at step 250: 88.767976
Minibatch accuracy: 93.0%
Validation accuracy: 89.0%
Minibatch loss at step 300: 84.449828
Minibatch accuracy: 93.0%
Validation accuracy: 89.5%
Minibatch loss at step 350: 80.343533
Minibatch accuracy: 93.8%
Validation accuracy: 90.2%
Minibatch loss at step 400: 76.513585
Minibatch accuracy: 90.6%
Validation accuracy: 90.0%
Minibatch loss at step 450: 72.712242
Minibatch accuracy: 93.0%
Validation accuracy: 89.9%
Test accuracy: 91.7%


In [94]:
regul_val = [pow(10, i) for i in np.arange(-4, -2, 0.1)]
accuracy_val = []

for regul in regul_val:
    offset = int(test_end_index)
    trainNormalX = rescaledNormalX[test_end_index:, :]
    class_batch_size = int(batch_size)/2
    
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        for step in range(num_steps):
            np.random.shuffle(trainNormalX)
            train_batch_data = np.concatenate((rescaledBotnetX[offset:(offset+class_batch_size),:], 
                                   trainNormalX[:class_batch_size,:]))
            train_batch_labels = np.concatenate((botnetY[:class_batch_size], 
                                 normalY[:class_batch_size]))
            train_batch_labels = (np.arange(num_labels) == train_batch_labels[:,None]).astype(np.float64)
            offset = offset + class_batch_size
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : train_batch_data, 
                         tf_train_labels : train_batch_labels,
                         beta_regul : regul
                        }
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
        accuracy_val.append(accuracy(test_prediction.eval(), test_labels))
        
print(accuracy_val)    

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:13: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:15: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:16: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[90.700000000000003, 91.900000000000006, 92.099999999999994, 91.099999999999994, 89.099999999999994, 91.799999999999997, 92.0, 90.5, 91.400000000000006, 91.299999999999997, 89.400000000000006, 89.099999999999994, 90.599999999999994, 88.200000000000003, 88.0, 86.900000000000006, 86.799999999999997, 88.599999999999994, 77.299999999999997, 86.5]


In [95]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.semilogx(regul_val, accuracy_val)
plt.grid(True)
plt.title('Test accuracy by regularization (1-layer net)')
plt.show()